In [5]:
import numpy as np
from utils import Func

In [27]:
def objective(x):
    """Функция цели (вес салата)"""
    return -(x[0] + x[1] + x[2])

objective = Func(lambda x: -(x[0] + x[1] + x[2]), 
                 lambda x: np.array((-1, -1, -1)))

penalty1 = Func(lambda x: max(0, 10*x[0] + x[1] + 8*x[2] - 20), 
                lambda x: np.array((10, 1, 8)) if 10*x[0] + x[1] + 8*x[2] - 20 > 0 else np.array((0,0,0)))

penalty2 = Func(lambda x: max(0, -2*x[0] + x[1] ),
                lambda x: np.array((-2, 1, 0)) if -2*x[0] + x[1] > 0 else np.array((0,0,0)))

def penalty_3_grad(x):
    if -x[0] -x[1] + 2*x[2] > 0:
        return np.array((-1, -1, 2))
    elif -x[0] -x[1] + 2*x[2] < 0:
        return np.array((1, 1, -2))
    return np.array((0,0,0))

penalty3 = Func(lambda x: abs(2*x[2] - x[0] - x[1]),
                penalty_3_grad)

In [28]:
def auxiliary_func_call(x, r):
    """Вспомогательная функция"""
    return objective(x) + r * (penalty1(x) + penalty2(x) + penalty3(x))

def auxiliary_func_grad(x, r):
    """Вспомогательная функция"""
    return objective.grad(x) + r * (penalty1.grad(x) + penalty2.grad(x) + penalty3.grad(x))

auxiliary_func = Func(auxiliary_func_call, auxiliary_func_grad)

In [31]:
auxiliary_func.grad((2,5,1), 5)

array([44, 14, 29])

In [20]:
def steepest_descent(x0, r, eps=1e-6, max_iter=10000):
    """Метод наискорейшего спуска"""
    x = x0.astype(np.float64)  # Преобразование x в float64
    for i in range(max_iter):
        grad = gradient(x, r)
        if np.linalg.norm(grad) < eps:
            break
        
        # Выбор шага методом дробления
        alpha = 1
        while auxiliary_func(x - alpha*grad, r) >= auxiliary_func(x, r) - 0.5*alpha*np.linalg.norm(grad)**2:
            alpha /= 2
        
        x_new = x - alpha * grad  # Переменная для новой точки
        if np.linalg.norm(x - x_new) < eps:  # Проверка изменения x
            break
        x = x_new
    
    return x
